To scrap the diseases name and their corresponding link from the nhsinform medcial website.

In [1]:
import bs4
import requests
import pandas as pd
from tqdm import tqdm

In [2]:

# The base URL for NHS Inform's A to Z illness and conditions list
home = "https://www.nhsinform.scot/illnesses-and-conditions/a-to-z/"

# Make an HTTP GET request to fetch the content of the webpage
res = requests.get(home)

# Parse the HTML content of the page using BeautifulSoup and the html5lib parser
soup = bs4.BeautifulSoup(res.text, 'html5lib')

# Print the parsed HTML content
print(soup)


<!DOCTYPE html>
<html lang="en-GB"><head>
	<meta charset="utf-8"/>
	<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
	<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
	<meta content="width=device-width, initial-scale=1, maximum-scale=2" name="viewport"/>
  <link href="https://appnhs24wp41a8c38064.blob.core.windows.net/blobappnhs24wp41a8c38064/wp-content/themes/nhsinform/assets/css/global.css" rel="stylesheet"/>
  <!-- <meta http-equiv="Content-Security-Policy"  content="script-src 'unsafe-inline'; style-src 'unsafe-inline'"> -->
  <meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots"/>

	<!-- This site is optimized with the Yoast SEO plugin v22.5 - https://yoast.com/wordpress/plugins/seo/ -->
	<title>A to Z list of common illnesses and conditions | NHS inform</title>
	<meta content="A to Z list of common illnesses and conditions - including their symptoms, causes and treatments" name="description"/>

In [3]:
# s = soup.find_all(attrs = { 'class' : 'az_list_indivisual' })

# extraction of all link tag from the content
s = soup.find_all('a', )

In [4]:
s

[<a class="navbar-brand" href="https://www.nhsinform.scot/">
                                 <img alt="NHS 24 logo in white with clear background" src="https://appnhs24wp41a8c38064.blob.core.windows.net/blobappnhs24wp41a8c38064/wp-content/uploads/2023/03/logo.svg" title="Click here to navigate to the homepage."/>
 						                </a>,
 <a class="nav-link" href="/"><span class="menu-image-title-hide menu-image-title">Home</span><img alt="NHS inform home logo" class="menu-image menu-image-title-hide" decoding="async" height="19" src="https://appnhs24wp41a8c38064.blob.core.windows.net/blobappnhs24wp41a8c38064/wp-content/uploads/2023/06/home-nhs.png" width="20"/></a>,
 <a class="nav-link" href="https://www.nhsinform.scot/illnesses-and-conditions/">Illnesses and conditions</a>,
 <a class="nav-link" href="https://www.nhsinform.scot/symptoms-and-self-help/">Symptoms and self-help</a>,
 <a class="nav-link" href="https://www.nhsinform.scot/tests-and-treatments/">Tests and treatments</a>

In [53]:
#  This function is used to extract the name and their corresponding link of each diseases and save them in csv file
def get_alphabet_links(s):
    """
    returns:
    list = links for pages for each alphabet
    
    """
    
# Initialize an empty list to store the data
    data_list = []

    # Wrap your iterable with tqdm
    for item in tqdm(s, desc="Processing items"):
        link = item.get('href')
        name = item.text.strip()
        data_list.append({'name': name, 'link': link})

    # Create the DataFrame after the loop
    df_diseases = pd.DataFrame(data_list)


    # Save you data in given file location
    df_diseases.to_csv('nhs_diseases.csv', index=False)
    print("Done!!!!")
    
    return

In [54]:
# function call to extract the useful data from content
get_alphabet_links(s)

Processing items: 100%|██████████| 147/147 [00:00<00:00, 94710.09it/s]

Done!!!!


To scrap the main component and details of each diseases from their corresponding links.

In [3]:
import bs4
import requests
import pandas as pd
import re
import argparse
from tqdm import tqdm

In [5]:
# Load the name and link data in pandas dataframe

df_diseases = pd.read_csv("nhs_diseases.csv")

In [19]:
# drop duplicate links from the dataset as multiple diseases can map to the same page
# df_diseases.drop_duplicates(subset="link", keep="last", inplace=True)

In [20]:
len(df_diseases)

12

In [21]:
# Define a dictionary to specify a user-agent header.
# The "User-Agent" header is a way for the client (in this case, your Python script) to tell the server what kind of software is making the request.
# This user-agent string is mimicking a web browser (Google Chrome) running on a Windows machine.
agent = {"User-Agent": 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'
}


In [22]:
class basic_diseases_data:
    """
    Scrape basic features for given diseases.
    Basic features include Symptoms, Overview, Risk factors, and Causes.
    """

    def __init__(self, url, soup_obj):
        # Initialize the class with the URL and BeautifulSoup object
        self.url = url
        self.soup_obj = soup_obj

    def get_basic_components(self, component):
        """
        Scrapes the basic components.
        Takes a component (e.g., Symptoms, Overview) as an argument and scrapes the related data.
        """
        # Find all h2 tags, which are typically section headers
        headings = self.soup_obj.find_all("h2")
        if headings is None:
            return 'NA'  # If no h2 tags are found, return 'NA'
        
        # Store the content systematically in a list of dictionaries
        content = []
        for heading in headings:
            # Initialize a dictionary to store the heading and associated content
            head = heading.text.strip()  # Extract and clean the heading text
            
            # Get the next sibling after the h2 tag (this could be paragraphs, lists, etc.)
            sibling = heading.find_next_sibling()

            data = ""  # Initialize a string to hold the content for the section
            
            # Loop through all the following siblings until we hit another h2 or the end of the relevant section
            while sibling and sibling.name not in ['h2', 'h3']:
                # If the sibling is a paragraph, add its text to the data string
                if sibling.name == 'p':
                    data += (" " + sibling.text.strip())
                # If the sibling is an unordered or ordered list, add its list items to the data string
                elif sibling.name in ['ul', 'ol']:
                    list_items = sibling.find_all('li')
                    data += (' '.join([li.text.strip() for li in list_items]))
                
                # Move to the next sibling in the HTML tree
                sibling = sibling.find_next_sibling()

            # If data was collected, store it in the content list
            if data:
                section = {
                    head: data  # Create a dictionary with the heading as the key and the content as the value
                }
                
                # Append the section dictionary to the content list
                content.append(section)

        return content  # Return the collected content as a list of dictionaries


In [23]:
# Enable the progress bar for pandas operations using tqdm
tqdm.pandas()

# Apply the scraping function to each link in the 'link' column and store the results in the 'About' column
df_diseases["About"] = df_diseases['link'].progress_apply(lambda x:
                                                        basic_diseases_data(x, bs4.BeautifulSoup(requests.get(
                                                            x, headers=agent).text, 'html5lib')).get_basic_components("About"),
                                                        )

# Print a confirmation message once the scraping process is complete
print("Scraped " + "About" + " !!")

100%|██████████| 12/12 [00:14<00:00,  1.24s/it]

Scraped About !!


In [24]:
# save dataframe
df_diseases.to_csv("nhs_disease_components.csv", index=False)